# A CNN predictor using the latent layer of a Convolutional Autoencoder (CAE) 
A neural network architecture where the compressed representation (latent layer) generated by a CAE is used as input for a Convolutional Neural Network (CNN) designed for prediction tasks, such as classification or regression. This approach leverages the CAE's ability to distill essential features from the input data, enhancing the performance of the subsequent prediction model.
## Architecture:
Outline of the combined architecture:
+ **CAE Encoder:** Input Image -> Convolutional Layers  -> Latent Space
+ **Latent Space Representation:** Pooling the Latent Space Layers
+ **CNN Predictor:** Latent Space -> Deconvolutional Layers -> Prediction

In [1]:
import tensorflow as tf

import numpy as np
from tensorflow.keras import layers, losses

from Models import CNN_pred_MMUMM_v01 as pcnn
cnn_slayers = pcnn.slayers
cnn_nlayers = pcnn.nlayers
cnn_version = pcnn.version

In [2]:
from platform import python_version
import sys
print('Python: ' + python_version()) # Python: 3.10.9
print('numpy: ' + np.__version__) # numpy: 1.23.5
print ('tensorflow: ' + sys.modules["tensorflow"].__version__) # tensorflow: 2.10.0

Python: 3.10.9
numpy: 1.23.5
tensorflow: 2.10.0


In [42]:
mask = np.array([[[[100]]*360]*50 + [[[0]]*360]*80 + [[[100]]*360]*50])

In [44]:
def loss(true, pred):
    error = tf.math.reduce_mean(tf.math.square(true - pred)* mask)
    return error

In [46]:
# cnn = CNN_pred()
cnn = pcnn.CNN_pred()
# cnn.compile(optimizer='adam', loss=losses.MeanSquaredError())
cnn.compile(optimizer='adam', loss=loss)

In [48]:
cnn.build((None,45, 90, cnn_nlayers))
cnn.model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_transpose_16 (Conv2D  (None, 45, 90, 8)        368       
 Transpose)                                                      
                                                                 
 conv2d_transpose_17 (Conv2D  (None, 90, 180, 16)      1168      
 Transpose)                                                      
                                                                 
 conv2d_transpose_18 (Conv2D  (None, 90, 180, 16)      2320      
 Transpose)                                                      
                                                                 
 conv2d_transpose_19 (Conv2D  (None, 180, 360, 8)      1160      
 Transpose)                                                      
                                                                 
 conv2d_4 (Conv2D)           (None, 180, 360, 1)      

In [ ]:
version = "01"
cnn = pcnn.CNN_pred()
cnn.model.load_weights('./Weights/cnn_pred_' + cnn_slayers + '_v' + cnn_version + '_weights_' + version)
cnn.compile(optimizer='adam', loss=losses.MeanSquaredError())
cnn.build((None,45, 90, cnn_nlayers))

In [22]:
# data_precip_x = np.load("../data/WaterPrecip_datacube_CNN_x_encoded.npy")
data_precip_x = np.load("../data/WaterPrecip_datacube_CNN_x_encoded_singles_masked.npy")
data_precip_y = np.load("../data/WaterPrecip_datacube_CNN_y_s2.npy")

In [26]:
print(data_precip_x.shape)
print(data_precip_y.shape)

(8826, 45, 90, 5)
(8826, 180, 360, 1)


In [24]:
nTest = 1000
iTest = np.random.choice(data_precip_x.shape[0],nTest, replace=False)
iTrain = [i for i in range(data_precip_x.shape[0]) if i not in iTest]

In [50]:
cnn.fit(data_precip_x[iTrain][:,:,:,[0,1,2,3,4]], data_precip_y[iTrain,:,:,:],
                epochs=10,
                shuffle=True,
                validation_data=(data_precip_x[iTest][:,:,:,[0,1,2,3,4]], data_precip_y[iTest,:,:,:]))

Epoch 1/10
245/245 [==============================] - 66s 267ms/step - loss: 629.1845 - val_loss: 604.9081
Epoch 2/10
245/245 [==============================] - 70s 286ms/step - loss: 591.9406 - val_loss: 586.1635
Epoch 3/10
245/245 [==============================] - 62s 251ms/step - loss: 580.6876 - val_loss: 577.8719
Epoch 4/10
245/245 [==============================] - 67s 274ms/step - loss: 576.1962 - val_loss: 576.6320
Epoch 5/10
245/245 [==============================] - 63s 256ms/step - loss: 573.8033 - val_loss: 573.8113
Epoch 6/10
245/245 [==============================] - 69s 282ms/step - loss: 571.7773 - val_loss: 572.2648
Epoch 7/10
245/245 [==============================] - 71s 289ms/step - loss: 570.7818 - val_loss: 571.0503
Epoch 8/10
245/245 [==============================] - 68s 277ms/step - loss: 569.8599 - val_loss: 569.7554
Epoch 9/10
245/245 [==============================] - 61s 250ms/step - loss: 568.6092 - val_loss: 568.3555
Epoch 10/10
245/245 [================

In [51]:
version = "02"
cnn.model.save_weights('./Weights/cnn_pred_' + cnn_slayers + '_v' + cnn_version + '_weights_' + version)